In [51]:
import pandas as pd
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

from pathlib import Path
import numpy as np

import plotly as px


In [53]:
def create_dataframes(folder_path):

    folder_path = Path(folder_path)

    for subdir in folder_path.iterdir():
        if subdir.is_dir():
            for csv_file in subdir.glob('*.csv'):
                df_name = str(csv_file.stem)
                
                df = pd.read_csv(csv_file,encoding='latin-1')
                globals() [df_name] = df
                
create_dataframes('/Users/jallison/All_Py/FoodSecurityAfrica')


df_temperature = Environment_Temperature_change_E_Africa


#### 0. Data Preprocessing

In [56]:
df_temperature = df_temperature.fillna(0)
df_temperature.drop(df_temperature[df_temperature['Element']=='Standard Deviation'].index,inplace=True)

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
df_temperature = df_temperature[df_temperature['Months'].isin(months)].groupby(['Area']).sum()
df_temperature.drop(columns=['Area Code','Months Code','Element Code'],inplace=True)
df_temperature.columns = df_temperature.columns.str[1:]

df_temperature.reset_index(inplace=True)
df = pd.melt(df_temperature,'Area')
df.sort_values(['Area','variable'],inplace=True)
df.rename(columns={'variable':'Year','value':'TempChange'},inplace=True)
df.reset_index(drop=True,inplace=True)


#### 1. Visualization

In [73]:
# Creating an interactive map using Plotly
fig = px.choropleth(df, 
                    locations='Area', 
                    locationmode='country names',
                    color='TempChange', 
                    animation_frame='Year',
                    color_continuous_scale='reds',
                    range_color=[df['TempChange'].min(), df['TempChange'].max()])

fig.update_layout(title_text='African Countries Temperature Change per Year (°C)')

# Displaying the map
fig.show()
